In [3]:
import sys
import os

# Path to the folder containing ipyformkit (one level up from examples/)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")
# Try importing now
import ipyformkit

Project root added to sys.path: C:\Users\Richard\AppData\Local\Programs


ModuleNotFoundError: No module named 'ipyformkit'